In [1]:
import datetime
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_relinc = pd.read_csv('data/relinc.csv')
df_relinc.head()

,religion,<10k,10-20k,20-30k,30-40k,40-50k,50-75k,75-100k,100-150k,>150k,refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,refused,15,14,15,11,10,35,21,17,18,116


In [3]:
# Applying melt (to a long format)
df_relinc = df_relinc.melt(id_vars=['religion'],var_name=['income'],value_name='frequency')
df_relinc.head()

,religion,income,frequency
0,Agnostic,<10k,27
1,Atheist,<10k,12
2,Buddhist,<10k,27
3,Catholic,<10k,418
4,refused,<10k,15


In [4]:
# Applying pivot_table (to a wide format)
df_relinc = (df_relinc.pivot_table(index = 'religion', columns = 'income', values = 'frequency')
   .reset_index() 
   .rename_axis(None, axis = 1))
df_relinc.head()

,religion,10-20k,100-150k,20-30k,30-40k,40-50k,50-75k,75-100k,<10k,>150k,refused
0,Agnostic,34,109,60,81,76,137,122,27,84,96
1,Atheist,27,59,37,52,35,70,73,12,74,76
2,Buddhist,21,39,30,34,33,58,62,27,53,54
3,Catholic,617,792,732,670,638,1116,949,418,633,1489
4,Evangelical Prot,869,723,1064,982,881,1486,949,575,414,1529


In [5]:
df_tb = pd.read_csv('data/tb.csv')
df_tb.columns

Index(['iso2', 'year', 'new_sp', 'new_sp_m04', 'new_sp_m514', 'new_sp_m014',
       'new_sp_m1524', 'new_sp_m2534', 'new_sp_m3544', 'new_sp_m4554',
       'new_sp_m5564', 'new_sp_m65', 'new_sp_mu', 'new_sp_f04', 'new_sp_f514',
       'new_sp_f014', 'new_sp_f1524', 'new_sp_f2534', 'new_sp_f3544',
       'new_sp_f4554', 'new_sp_f5564', 'new_sp_f65', 'new_sp_fu'],
      dtype='object')

In [6]:
df_tb.tail()

,iso2,year,new_sp,new_sp_m04,new_sp_m514,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,...,new_sp_f04,new_sp_f514,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sp_fu
5764,ZW,2004,14581.0,NaN,NaN,187.0,833.0,2908.0,2298.0,1056.0,...,NaN,NaN,225.0,1140.0,2858.0,1565.0,622.0,214.0,111.0,NaN
5765,ZW,2005,13155.0,NaN,NaN,210.0,837.0,2264.0,1855.0,762.0,...,NaN,NaN,269.0,1136.0,2242.0,1255.0,578.0,193.0,603.0,NaN
5766,ZW,2006,12718.0,NaN,NaN,215.0,736.0,2391.0,1939.0,896.0,...,NaN,NaN,237.0,1020.0,2424.0,1355.0,632.0,230.0,96.0,NaN
5767,ZW,2007,10583.0,6.0,132.0,138.0,500.0,3693.0,0.0,716.0,...,7.0,178.0,185.0,739.0,3311.0,0.0,553.0,213.0,90.0,NaN
5768,ZW,2008,9830.0,NaN,NaN,127.0,614.0,0.0,3316.0,704.0,...,NaN,NaN,145.0,840.0,0.0,2890.0,467.0,174.0,105.0,0.0


In [7]:
# Applying melt (to a long format)
df_tb = df_tb.melt(id_vars=['iso2','year'],var_name=['demographic'],value_name='cases')
df_tb.sample(5)

,iso2,year,demographic,cases
79252,PS,1983,new_sp_f014,NaN
121035,YE,2005,new_sp_fu,NaN
84117,MM,2007,new_sp_f1524,2719.0
82425,FI,1986,new_sp_f1524,NaN
92563,AS,1994,new_sp_f3544,NaN


In [8]:
# Creating new columns for sex and age
df_tb = (df_tb.assign(
sex = lambda x: x.demographic.str[0].astype(str),
age = lambda x: x.demographic.str[1:].astype(str))
      .drop('demographic',axis=1))
df_tb.sample(5)

,iso2,year,cases,sex,age
78054,LU,1984,NaN,n,ew_sp_f014
78358,MN,1988,NaN,n,ew_sp_f014
50567,RO,2005,952.0,n,ew_sp_m5564
68832,US,1998,NaN,n,ew_sp_f04
32280,MP,2007,0.0,n,ew_sp_m2534


In [9]:
# Styling the dataset
df_tb.update(pd.DataFrame({'age':[age[:2]+'-'+age[2:] if len(age) == 4 else (age) for age in df_tb['age']]}))
df_tb=(df_tb.replace(to_replace =['m','f','014','65','u'],value =['Male','Female','0-14','65+','unknown'])
            .dropna())
df_tb.sample(10)

,iso2,year,cases,sex,age
93342,CI,2005,989.0,n,ew_sp_f3544
27210,PG,2005,183.0,n,ew_sp_m1524
20628,ML,2006,28.0,n,ew_sp_m014
3209,MD,1998,477.0,n,ew_sp
106645,KW,2008,5.0,n,ew_sp_f5564
30746,GH,2002,1245.0,n,ew_sp_m2534
79767,SN,2003,77.0,n,ew_sp_f014
42794,IR,1997,304.0,n,ew_sp_m4554
102203,PG,2001,17.0,n,ew_sp_f4554
3982,NZ,1998,106.0,n,ew_sp


In [10]:
df_weather = pd.read_csv('data/weather-raw.csv')
df_weather.sample(5)

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
6,MX17004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MX17004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN


In [11]:
# Applying melt (to a long format)
df_weather = df_weather.melt(id_vars=['id','year','month','element'],var_name=['day'],value_name='temp')
df_weather.update(pd.DataFrame({'day':[day[1:] for day in df_weather['day']]}))
df_weather.sample(5)

,id,year,month,element,day,temp
41,MX17004,2010,1,tmin,5,NaN
36,MX17004,2010,4,tmax,4,NaN
15,MX17004,2010,3,tmin,2,NaN
74,MX17004,2010,3,tmax,8,NaN
24,MX17004,2010,3,tmax,3,NaN


In [12]:
# applying pivot_table to create columns for tmin and tmax
df_weather = (df_weather.pivot_table(index = ['year','month','day','id'], columns = 'element', values = 'temp')
       .reset_index().rename_axis(None, axis = 1))
df_weather

,year,month,day,id,tmax,tmin
0,2010,2,2,MX17004,NaN,14.4
1,2010,2,2,MX17004,27.3,NaN
2,2010,2,3,MX17004,NaN,14.4
3,2010,2,3,MX17004,24.1,NaN
4,2010,3,5,MX17004,32.1,14.2


In [13]:
# Creating a date column
df_weather = (df_weather.assign(date = lambda x: x.year.astype('str') +'/'+ x.month.astype('str').str.zfill(2) +'/'+ x.day.astype('str').str.zfill(2))
            .drop(['year', 'month','day'],axis=1))
df_weather['date'] =  pd.to_datetime(df_weather['date'], format='%Y/%m/%d')
# Grouping by date
df_weather = (df_weather.filter(['date', 'tmax','tmin']).groupby('date').aggregate('mean').reset_index())
df_weather

,date,tmax,tmin
0,2010-02-02,27.3,14.4
1,2010-02-03,24.1,14.4
2,2010-03-05,32.1,14.2


In [14]:
df_bill = pd.read_csv('data/billboard.csv', header=0, encoding = 'unicode_escape')
df_bill.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_bill.columns

Index(['year', 'artist.inverted', 'track', 'time', 'genre', 'date.entered',
       'date.peaked', 'x1st.week', 'x2nd.week', 'x3rd.week', 'x4th.week',
       'x5th.week', 'x6th.week', 'x7th.week', 'x8th.week', 'x9th.week',
       'x10th.week', 'x11th.week', 'x12th.week', 'x13th.week', 'x14th.week',
       'x15th.week', 'x16th.week', 'x17th.week', 'x18th.week', 'x19th.week',
       'x20th.week', 'x21st.week', 'x22nd.week', 'x23rd.week', 'x24th.week',
       'x25th.week', 'x26th.week', 'x27th.week', 'x28th.week', 'x29th.week',
       'x30th.week', 'x31st.week', 'x32nd.week', 'x33rd.week', 'x34th.week',
       'x35th.week', 'x36th.week', 'x37th.week', 'x38th.week', 'x39th.week',
       'x40th.week', 'x41st.week', 'x42nd.week', 'x43rd.week', 'x44th.week',
       'x45th.week', 'x46th.week', 'x47th.week', 'x48th.week', 'x49th.week',
       'x50th.week', 'x51st.week', 'x52nd.week', 'x53rd.week', 'x54th.week',
       'x55th.week', 'x56th.week', 'x57th.week', 'x58th.week', 'x59th.week',
       '

In [16]:
# Applying melt (to a long format)
df_bill = (df_bill.melt(id_vars=['year','artist.inverted','track','genre','date.entered','date.peaked','time'],var_name=['week'],value_name='rank'))
# Week to number 
df_bill.update(pd.DataFrame({'week': np.ravel([list(map(int, re.findall(r'\d+', i))) for i in df_bill['week']])}))
df_bill.head()

,year,artist.inverted,track,genre,date.entered,date.peaked,time,week,rank
0,2000,Destiny's Child,Independent Women Part I,Rock,2000-09-23,2000-11-18,3:38,1,78.0
1,2000,Santana,"Maria, Maria",Rock,2000-02-12,2000-04-08,4:18,1,15.0
2,2000,Savage Garden,I Knew I Loved You,Rock,1999-10-23,2000-01-29,4:07,1,71.0
3,2000,Madonna,Music,Rock,2000-08-12,2000-09-16,3:45,1,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),Rock,2000-08-05,2000-10-14,3:38,1,57.0


In [17]:
df_bill.dtypes

year                 int64
artist.inverted     object
track               object
genre               object
date.entered        object
date.peaked         object
time                object
week                object
rank               float64
dtype: object

In [18]:
# creating a date column from date.entered and week
df_bill['date.entered'] = pd.to_datetime(df_bill['date.entered'], format='%Y-%m-%d')
df_bill = (df_bill.assign(date=[df_bill['date.entered'][i]+datetime.timedelta(weeks = df_bill['week'][i]-1) for i in range(len(df_bill['week']))])
         .drop(['date.entered','date.peaked','week'], axis=1)
         .sort_values('artist.inverted', ascending=True)
         .reset_index(drop=True))
df_bill.head()

,year,artist.inverted,track,genre,time,rank,date
0,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),Rap,4:22,NaN,2000-11-25
1,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),Rap,4:22,NaN,2001-07-21
2,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),Rap,4:22,NaN,2000-09-16
3,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),Rap,4:22,NaN,2001-02-24
4,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),Rap,4:22,NaN,2000-12-16


In [19]:
# creating an id column from track
labels,unique = pd.factorize(list(df_bill['track']))
df_bill['id']=labels+1
df_bill.reset_index(drop=True)
df_bill.head()

,year,artist.inverted,track,genre,time,rank,date,id
0,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),Rap,4:22,NaN,2000-11-25,1
1,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),Rap,4:22,NaN,2001-07-21,1
2,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),Rap,4:22,NaN,2000-09-16,1
3,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),Rap,4:22,NaN,2001-02-24,1
4,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),Rap,4:22,NaN,2000-12-16,1


In [20]:
# creating a new dataframe for rank
df_rank = df_bill.filter(['id', 'date', 'rank']).dropna()
df_rank = df_rank.sort_values(by=['id','date']).reset_index(drop=True)
df_rank.head()

,id,date,rank
0,1,2000-02-26,87.0
1,1,2000-03-04,82.0
2,1,2000-03-11,72.0
3,1,2000-03-18,77.0
4,1,2000-03-25,87.0


In [21]:
# creating a new dataframe for song
df_song = df_bill.filter(['id', 'artist.inverted', 'track','time'])
df_song = df_song.drop_duplicates('id').reset_index(drop=True)
df_song.head()

,id,artist.inverted,track,time
0,1,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22
1,2,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Back Your Stuff),3:15
2,3,3 Doors Down,Loser,4:24
3,4,3 Doors Down,Kryptonite,3:53
4,5,504 Boyz,Wobble Wobble,3:35


In [22]:
df_baby14 = pd.read_csv('data/2014-baby-names-illinois.csv')
df_baby15 = pd.read_csv('data/2015-baby-names-illinois.csv')
df_baby14.head()

,rank,name,frequency,sex
0,1,Noah,837,Male
1,2,Alexander,747,Male
2,3,William,687,Male
3,4,Michael,680,Male
4,5,Liam,670,Male


In [23]:
df_baby15.head()

,rank,name,frequency,sex
0,1,Noah,863,Male
1,2,Liam,709,Male
2,3,Alexander,703,Male
3,4,Jacob,650,Male
4,5,William,618,Male


In [24]:
# Creating a column for the year
df_baby14['year']='2014'
df_baby15['year']='2015'
# Concatenating the datasets
df_baby = pd.concat([df_baby14, df_baby15]).sort_values(by=['rank'])
(df_baby.set_index('rank', inplace=True))
df_baby.head()

,name,frequency,sex,year
rank,,,,
1,Noah,837,Male,2014
1,Noah,863,Male,2015
2,Alexander,747,Male,2014
2,Liam,709,Male,2015
3,William,687,Male,2014
